# Import library

In [1]:
import pandas as pd
from my_example import obj_ic_fundacoes, tensao_adm_solo
from metapy_toolbox import metaheuristic_optimizer

# Setup and call function

In [2]:
# df_aux = pd.read_excel('input.xlsx')
# df = tensao_adm_solo(df_aux)
# df

In [3]:
import pandas as pd
import numpy as np

# Gerando dados aleatórios para o DataFrame
np.random.seed(42)  # Para reprodutibilidade

# Número de linhas
n = 4

# Criando o DataFrame
data = {
    "elemento": [f"Elemento_{i+1}" for i in range(n)],
    "xg (m)": np.round(np.random.uniform(0, 100, n), 2),
    "yg (m)": np.round(np.random.uniform(0, 100, n), 2),
    "spt": np.random.randint(5, 50, n),
    "solo": np.random.choice(["argila", "silte", "pedregulho", "areia"], n),
}

df_aux = pd.DataFrame(data)
df = tensao_adm_solo(df_aux)
df

,elemento,xg (m),yg (m),spt,solo,sigma_adm (kPa)
0,Elemento_1,37.45,15.60,40,silte,800.0
1,Elemento_2,95.07,15.60,44,argila,880.0
2,Elemento_3,73.20,5.81,28,silte,560.0
3,Elemento_4,59.87,86.62,7,areia,175.0


In [4]:
import random

# Função para gerar uma combinação de 3 valores float aleatórios
def gerar_combinacoes():
    return [round(random.uniform(0, 100), 2) for _ in range(3)]

# Número de pilares
n_pilares = n

# Gerando a lista com 4 dicionários
lista_dicionarios = [
    {
        "combinação 1": gerar_combinacoes(),
        "combinação 2": gerar_combinacoes(),
        "combinação 3": gerar_combinacoes(),
    }
    for _ in range(n_pilares)
]

lista_comb = lista_dicionarios.copy()
lista_comb

[{'combinação 1': [23.5, 85.8, 70.56],
  'combinação 2': [7.67, 12.69, 38.09],
  'combinação 3': [81.54, 5.77, 97.77]},
 {'combinação 1': [33.85, 15.57, 35.87],
  'combinação 2': [56.0, 64.24, 70.39],
  'combinação 3': [91.15, 0.18, 28.55]},
 {'combinação 1': [50.15, 96.48, 5.35],
  'combinação 2': [42.95, 58.86, 77.53],
  'combinação 3': [92.6, 34.07, 49.87]},
 {'combinação 1': [87.6, 64.45, 16.78],
  'combinação 2': [89.72, 40.48, 91.28],
  'combinação 3': [65.31, 20.39, 87.31]}]

In [5]:
dim_a = []
dim_b = []

for i, row in df.iterrows():
        # Recolhendo os dados do pilar
        dados_fundacao = { 
                                'combinações': lista_comb[i],
                                'sigma_adm (kPa)': row['sigma_adm (kPa)'],
                         }
        
        # Otimização
        algorithm_setup = {   
                'number of iterations': 100,
                'number of population': 5,
                'number of dimensions': 2,
                'x pop lower limit': [0.60, 0.60], # 0.60
                'x pop upper limit': [2.25, 2.25],
                'none variable': dados_fundacao,
                'objective function': obj_ic_fundacoes,
                'algorithm parameters': {
                                        'selection': {'type': 'roulette'},
                                        'crossover': {'crossover rate (%)': 82, 'type':'linear'},
                                        'mutation': {'mutation rate (%)': 12, 'type': 'hill climbing', 'cov (%)': 15, 'pdf': 'gaussian'},
                                        }
                }
        n_rep = 5
        general_setup = {   
                'number of repetitions': n_rep,
                'type code': 'real code',
                'initial pop. seed': [None] * n_rep,
                'algorithm': 'genetic_algorithm_01',
                }
        df_all_reps, df_resume_all_reps, reports, status = metaheuristic_optimizer(algorithm_setup, general_setup)
        df_novo = df_resume_all_reps[status]
        dimensoes_sapata_a = list(df_novo['X_0_BEST'])[-1]
        dimensoes_sapata_b = list(df_novo['X_1_BEST'])[-1]
        dim_a.append(dimensoes_sapata_a)
        dim_b.append(dimensoes_sapata_b)

# Atribuição da dimensão otimizada
df['dimensão a (m)'] = dim_a
df['dimensão b (m)'] = dim_b

 Optimization results: 

 - Best repetition id:    0
 - Best of:               6.7707964306e-01
 - Design variables:      [1.2040241970482808, 0.9372453431915359]
 - Process time (s):      6.483927
 - Best process time (s): 0.007270

 Optimization results: 

 - Best repetition id:    4
 - Best of:               5.9223871627e-01
 - Design variables:      [0.9314914173241081, 1.0596603562397093]
 - Process time (s):      5.777048
 - Best process time (s): 0.008420

 Optimization results: 

 - Best repetition id:    1
 - Best of:               7.9878122075e-01
 - Design variables:      [0.9995364575684245, 1.3319194357582598]
 - Process time (s):      5.453718
 - Best process time (s): 0.006861

 Optimization results: 

 - Best repetition id:    0
 - Best of:               1.7753806622e+00
 - Design variables:      [1.3150967868335075, 2.25]
 - Process time (s):      7.480420
 - Best process time (s): 0.003572



In [6]:
df

,elemento,xg (m),yg (m),spt,solo,sigma_adm (kPa),dimensão a (m),dimensão b (m)
0,Elemento_1,37.45,15.60,40,silte,800.0,1.204024,0.937245
1,Elemento_2,95.07,15.60,44,argila,880.0,0.931491,1.059660
2,Elemento_3,73.20,5.81,28,silte,560.0,0.999536,1.331919
3,Elemento_4,59.87,86.62,7,areia,175.0,1.315097,2.250000
